Zuerst importieren wir die benötigten Bibliotheken pandas und numpy für Datenanalyse und numerische Operationen. Anschließend lesen wir die Daten zu Niederschlag, Wind, Sonnenstunden und Temperatur aus den jeweiligen CSV-Dateien ein. Mit der head() Methode geben wir einen Überblick über die ersten Zeilen jedes Datensatzes.

In [4]:
import pandas as pd
import numpy as np
regen = pd.read_csv("./Desktop/Statistical Computing/Fallstudie/Daten_DWD/niederschlag/data/niederschlag.csv", sep=';')
wind = pd.read_csv("./Desktop/Statistical Computing/Fallstudie/Daten_DWD/mean_wind/data/wind.csv", sep=';')
sonne = pd.read_csv("./Desktop/Statistical Computing/Fallstudie/Daten_DWD/sonne/data/sonnenstd.csv", sep=';')
temp = pd.read_csv("./Desktop/Statistical Computing/Fallstudie/Daten_DWD/mean_temp/data/temp.csv", sep=';')
print(regen.head())
print(wind.head())
print(sonne.head())
print(temp.head())

   Unnamed: 0  Stadt       Datum  Regen  Unnamed: 4  Unnamed: 5  Unnamed: 6
0         NaN   1048  2013-10-03    0.0         NaN         NaN         NaN
1         NaN   1048  2013-10-04    0.0         NaN         NaN         NaN
2         NaN   1048  2013-10-05    0.0         NaN         NaN         NaN
3         NaN   1048  2013-10-06    0.0         NaN         NaN         NaN
4         NaN   1048  2013-10-07    0.0         NaN         NaN         NaN
   Unnamed: 0  Stadt       Datum  Wind  Unnamed: 4  Unnamed: 5  Unnamed: 6
0         NaN   1048  2013-10-03   6.2         NaN         NaN         NaN
1         NaN   1048  2013-10-04   8.7         NaN         NaN         NaN
2         NaN   1048  2013-10-05   6.4         NaN         NaN         NaN
3         NaN   1048  2013-10-06   1.6         NaN         NaN         NaN
4         NaN   1048  2013-10-07   1.6         NaN         NaN         NaN
   Unnamed: 0  Stadt       Datum  Sonnenstd  Unnamed: 4  Unnamed: 5  \
0         NaN   1048  2

Um die Datenqualität zu verbessern, entfernen wir zuerst alle leeren Spalten aus den Datensätzen regen, wind, sonne und temp. Dies wird mit der dropna() Methode erreicht. Anschließend speichern wir diese bereinigten Datensätze in einer Liste namens ds_list. Zum Schluss iterieren wir über jeden Datensatz in dieser Liste und drucken mit der info() Methode eine Zusammenfassung, um einen Überblick über die verbleibenden Daten und ihre Struktur zu erhalten.

In [2]:
regen = regen.dropna(axis=1, how='all')
wind = wind.dropna(axis=1, how='all')
sonne = sonne.dropna(axis=1, how='all')
temp = temp.dropna(axis=1, how='all')
ds_list = [regen,wind,sonne,temp]
for ds in ds_list:
    print(ds.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36482 entries, 0 to 36481
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Stadt   36482 non-null  int64  
 1   Datum   36482 non-null  object 
 2   Regen   36482 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 855.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36276 entries, 0 to 36275
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Stadt   36276 non-null  int64  
 1   Datum   36276 non-null  object 
 2   Wind    36276 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 850.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36311 entries, 0 to 36310
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Stadt      36311 non-null  int64  
 1   Datum      36311 non-null  object 
 2   Son

Für jeden Datensatz in ds_list extrahieren wir die einzigartigen Werte der Spalte Stadt und listen sie auf. Dies gibt uns eine Übersicht über die verschiedenen Städte, die in jedem der Datensätze vertreten sind.

In [3]:
for ds in ds_list:
    unique_cities = ds['Stadt'].unique()
    print(unique_cities)   

[1048 1443 1975 2564 2712 3086 4104 4271 5100  701]
[1048 1443 1975 2712 2961 3086 4104 4271 5100  701]
[1048 1443 1975 2712 2961 3086 4104 4271 5100  701]
[1048 1443 1975 2712 2961 3086 4104 4271 5100  701]


Wir ersetzen alle Zeilen der Stadt-ID 2564 durch 2961. Nach der Korrektur drucken wir die Werte der Spalte Stadt aus, um sicherzustellen, dass die Änderungen korrekt vorgenommen wurden und uns einen Überblick über die aktuell vertretenen Stadt-IDs zu geben.

In [4]:
regen.loc[regen["Stadt"]==2564,"Stadt"]=2961
print(regen["Stadt"].unique())


[1048 1443 1975 2961 2712 3086 4104 4271 5100  701]


Die Funktion combine_dataframes_for_city ermöglicht es, mehrere DataFrames anhand eines bestimmten Stadt-Codes (city_code) zu kombinieren.
Zunächst werden die DataFrames gefiltert, sodass nur die Zeilen übrig bleiben, die den angegebenen Stadt-Code haben. Diese gefilterten DataFrames werden dann der Reihe nach kombiniert. Die Kombination erfolgt auf Basis der Spalten Datum und Stadt. Wenn in einem der DataFrames ein Datum-Stadt-Paar fehlt, wird es dennoch im kombinierten DataFrame durch die Nutzung von how='outer' aufgenommen.

Das Ergebnis ist ein DataFrame, der alle Daten für die angegebene Stadt aus allen bereitgestellten DataFrames enthält.

In [5]:
def combine_dataframes_for_city(city_code, dfs):
    """
    Kombiniert mehrere DataFrames für einen spezifischen Stadtcode.
    
    """
    filtered_dfs = [df[df['Stadt'] == city_code] for df in dfs]
    
    combined_df = filtered_dfs[0]
    for df in filtered_dfs[1:]:
        combined_df = pd.merge(combined_df, df, on=['Datum', 'Stadt'], how='outer')
        
    return combined_df

Wir haben ein Dictionary namens city_codes, das verschiedene Städtenamen ihren entsprechenden Stadt-Codes zuordnet.

Mit einer Schleife gehen wir durch jedes "Stadt-Code-Paar" in diesem Dictionary. Für jeden Stadt-Code rufen wir die zuvor definierte Funktion combine_dataframes_for_city auf, um alle relevanten Daten aus den DataFrames in ds_list für die jeweilige Stadt zu kombinieren. Das kombinierte DataFrame für jede Stadt wird dann der Liste city_list hinzugefügt.

Am Ende drucken wir das kombinierte DataFrame für die erste Stadt in city_list als Beispiel und zur Überprüfung.

In [7]:
city_codes = {
    'trier': 5100,
    'hamburg': 1975,
    'kiel': 2961,
    'dresden': 1048,
    'regensburg': 4104,
    'rostock': 4271,
    'konstanz': 2712,
    'bremerhaven': 701,
    'luebeck': 3086,
    'freiburg': 1443
}

city_list = []

for city, code in city_codes.items():
    city = combine_dataframes_for_city(code, ds_list)
    city_list.append(city)

# Zum Zugriff auf einen spezifischen DataFrame:
print(city_list[0])

      Stadt       Datum  Regen  Wind  Sonnenstd  Temp
0      5100  2013-10-03    1.1   3.8      6.700  11.5
1      5100  2013-10-04   14.0   1.8      1.100  14.0
2      5100  2013-10-05    5.9   1.5      0.000  14.2
3      5100  2013-10-06    0.0   2.5      1.400  12.4
4      5100  2013-10-07    0.0   2.2      1.800  12.1
...     ...         ...    ...   ...        ...   ...
3647   5100  2023-10-02    0.0   1.6      9.367  16.9
3648   5100  2023-10-03    5.0   4.4      2.950  15.7
3649   5100  2017-06-27    NaN   2.9      1.000  18.4
3650   5100  2023-03-31    NaN   NaN      1.800  10.8
3651   5100  2023-04-01    NaN   NaN      0.100   9.3

[3652 rows x 6 columns]


Die Funktion get_city_name nimmt einen Stadt-Code als Eingabe und gibt den zugehörigen Städtenamen aus dem Dictionary city_codes zurück. Falls der Code nicht gefunden wird, gibt sie None zurück.

Im nächsten Abschnitt wird für jede Stadt in city_list sowohl der größte als auch der kleinste Wert für die Spalten Sonnenstd, Temp, Wind und Regen ermittelt. Diese Werte werden dann zusammen mit dem Städtenamen ausgedruckt. Dies gibt uns einen Überblick über die Extremwerte für verschiedene Wetterkriterien in jeder Stadt.

Schließlich wird für jeden kombinierten Datensatz in city_list die Methode info() aufgerufen. Dies gibt eine kurze Zusammenfassung des Datensatzes aus, einschließlich der Anzahl der Einträge, der Spalten und der Datenarten.

In [8]:
def get_city_name(code):
    for name, city_code in city_codes.items():
        if city_code == code:
            return name
    return None

for city in city_list:
    # Größter Wert
    max_value_sun = city['Sonnenstd'].max()
    max_value_temp = city['Temp'].max()
    max_value_wind = city['Wind'].max()
    max_value_rain = city['Regen'].max()
    
    # Kleinster Wert
    min_value_sun = city['Sonnenstd'].min()
    min_value_temp = city['Temp'].min()
    min_value_wind = city['Wind'].min()
    min_value_rain = city['Regen'].min()
    city_name = get_city_name(city['Stadt'].unique()[0])
    print("Stadt:", city_name)
    print("Größter Wert in Sonnenstd:", max_value_sun)
    print("Größter Wert in Temperatur:", max_value_temp)
    print("Größter Wert in Wind:", max_value_wind)
    print("Größter Wert in Regen:", max_value_rain)
    print("Kleinster Wert in Sonnenstd:", min_value_sun)
    print("Kleinster Wert in Temperatur:", min_value_temp)
    print("Kleinster Wert in Wind:", min_value_wind)
    print("Kleinster Wert in Regen:", min_value_rain)

for city_info in city_list:
    print(city_info.info())

Stadt: trier
Größter Wert in Sonnenstd: 967.0
Größter Wert in Temperatur: 30.1
Größter Wert in Wind: 13.6
Größter Wert in Regen: 63.9
Kleinster Wert in Sonnenstd: 0.0
Kleinster Wert in Temperatur: -7.6
Kleinster Wert in Wind: 0.8
Kleinster Wert in Regen: 0.0
Stadt: hamburg
Größter Wert in Sonnenstd: 983.0
Größter Wert in Temperatur: 27.7
Größter Wert in Wind: 11.4
Größter Wert in Regen: 43.1
Kleinster Wert in Sonnenstd: 0.0
Kleinster Wert in Temperatur: -10.3
Kleinster Wert in Wind: 0.6
Kleinster Wert in Regen: 0.0
Stadt: kiel
Größter Wert in Sonnenstd: 983.0
Größter Wert in Temperatur: 25.7
Größter Wert in Wind: 22.4
Größter Wert in Regen: 37.7
Kleinster Wert in Sonnenstd: 0.0
Kleinster Wert in Temperatur: -8.0
Kleinster Wert in Wind: 1.6
Kleinster Wert in Regen: 0.0
Stadt: dresden
Größter Wert in Sonnenstd: 983.0
Größter Wert in Temperatur: 29.9
Größter Wert in Wind: 12.0
Größter Wert in Regen: 56.2
Kleinster Wert in Sonnenstd: 0.0
Kleinster Wert in Temperatur: -12.3
Kleinster Wert i

Für jede Stadt in city_list berechnen wir den Medianwert der Spalte Sonnenstd. Anschließend aktualisieren wir die Werte in der Spalte Sonnenstd: Wenn ein Wert größer als 18 Stunden ist (was für einen Tag ungewöhnlich wäre), ersetzen wir diesen durch den zuvor berechneten Medianwert. Dies könnte dazu dienen, Ausreißer oder fehlerhafte Datenpunkte zu korrigieren, indem sie durch einen repräsentativeren Wert (den Median) ersetzt werden.



In [2]:
for city in city_list:
    median_value = city['Sonnenstd'].median()
    city['Sonnenstd'] = city['Sonnenstd'].apply(lambda x: median_value if x > 18 else x)

NameError: name 'city_list' is not defined

Nach der Korrektur der Werte in der Spalte "Sonnenstd" ist es wichtig zu überprüfen, ob sich die extremen Werte in unseren Datensätzen geändert haben.

In [12]:

for city in city_list:
    # Größter Wert
    max_value_sun = city['Sonnenstd'].max()
    max_value_temp = city['Temp'].max()
    max_value_wind = city['Wind'].max()
    max_value_rain = city['Regen'].max()
    
    # Kleinster Wert
    min_value_sun = city['Sonnenstd'].min()
    min_value_temp = city['Temp'].min()
    min_value_wind = city['Wind'].min()
    min_value_rain = city['Regen'].min()
    city_name = get_city_name(city['Stadt'].unique()[0])
    print("Stadt:", city_name)
    print("Größter Wert in Sonnenstd:", max_value_sun)
    print("Größter Wert in Temperatur:", max_value_temp)
    print("Größter Wert in Wind:", max_value_wind)
    print("Größter Wert in Regen:", max_value_rain)
    print("Kleinster Wert in Sonnenstd:", min_value_sun)
    print("Kleinster Wert in Temperatur:", min_value_temp)
    print("Kleinster Wert in Wind:", min_value_wind)
    print("Kleinster Wert in Regen:", min_value_rain)

Stadt: trier
Größter Wert in Sonnenstd: 17.0
Größter Wert in Temperatur: 30.1
Größter Wert in Wind: 13.6
Größter Wert in Regen: 63.9
Kleinster Wert in Sonnenstd: 0.0
Kleinster Wert in Temperatur: -7.6
Kleinster Wert in Wind: 0.8
Kleinster Wert in Regen: 0.0
Stadt: hamburg
Größter Wert in Sonnenstd: 17.0
Größter Wert in Temperatur: 27.7
Größter Wert in Wind: 11.4
Größter Wert in Regen: 43.1
Kleinster Wert in Sonnenstd: 0.0
Kleinster Wert in Temperatur: -10.3
Kleinster Wert in Wind: 0.6
Kleinster Wert in Regen: 0.0
Stadt: kiel
Größter Wert in Sonnenstd: 17.0
Größter Wert in Temperatur: 25.7
Größter Wert in Wind: 22.4
Größter Wert in Regen: 37.7
Kleinster Wert in Sonnenstd: 0.0
Kleinster Wert in Temperatur: -8.0
Kleinster Wert in Wind: 1.6
Kleinster Wert in Regen: 0.0
Stadt: dresden
Größter Wert in Sonnenstd: 17.0
Größter Wert in Temperatur: 29.9
Größter Wert in Wind: 12.0
Größter Wert in Regen: 56.2
Kleinster Wert in Sonnenstd: 0.0
Kleinster Wert in Temperatur: -12.3
Kleinster Wert in Wi

Die Funktion clean_data dient dazu, die Wetterdaten jedes Stadt-DataFrames zu bereinigen und zu standardisieren:

Datumsformatierung: Das Datum wird in ein standardisiertes Format (YYYY-MM-DD) umgewandelt.
Vollständigkeit des Datums: Der Code stellt sicher, dass das DataFrame genau 3653 Zeilen hat, die den gesamten Zeitraum von '2013-10-03' bis '2023-10-03' abdecken. Falls bestimmte Datenpunkte fehlen, werden sie mit entsprechenden Zeilen (und NaN-Werten) ergänzt.
Behandlung von fehlenden Werten: Für die metrischen Spalten Regen, Wind, Sonnenstd und Temp werden fehlende Werte (NaN) durch den Durchschnittswert dieser Spalte ersetzt. (Ein alternativer Ansatz, der im Code aber auskommentiert ist, würde zufällige Werte zwischen dem Minimal- und Maximalwert der Spalte für fehlende Werte verwenden.)
Nachdem die Bereinigungsfunktion definiert wurde, wird sie auf jedes Stadt-DataFrame in city_list angewendet. Die bereinigten Daten werden dann in einer neuen Liste, cleaned_city_list, gespeichert.

Kopieren Sie diesen Text in eine Markdown-Zelle in Ihrem Jupyter Notebook, um den bereitgestellten Codeabschnitt zu erläutern.

In [13]:

def clean_data(df, start_date='2013-10-03', end_date='2023-10-03'):
    # Datum in standardisiertes Format umwandeln
    
    df['Datum'] = pd.to_datetime(df['Datum'])

    # Stellen Sie sicher, dass das DataFrame 3653 Zeilen hat, die den gesamten Datumsbereich abdecken
    full_date_range = pd.date_range(start=start_date, end=end_date)
    df = df.set_index('Datum').reindex(full_date_range).reset_index().rename(columns={'index': 'Datum'})

    # Für die anderen metrischen Spalten
    for col in ['Regen', 'Wind', 'Sonnenstd', 'Temp']:
        # NaN-Werte durch den Mittelwert ersetzen
        
            mean_value = df[col].mean()
            df[col].fillna(mean_value, inplace=True)
        
        # Alternativ: Zufällige Werte zwischen Min und Max für NaN Werte ersetzen
            #min_value = df[col].min()
            #max_value = df[col].max()
            #df[col] = df[col].apply(lambda x: np.random.uniform(min_value, max_value) if pd.isnull(x) else x)

    return df

# Die aktualisierte Liste
cleaned_city_list = []

for city_df in city_list:
    cleaned_city_list.append(clean_data(city_df))

Nachdem die Bereinigungsfunktion auf jeden Stadt-DataFrame angewendet wurde und die bereinigten Daten in cleaned_city_list gespeichert sind, nutzen wir den folgenden Codeabschnitt, um die ersten und die letzten fünf Zeilen jedes bereinigten Stadt-DataFrames zu überprüfen.

In [14]:


# Verwenden Sie diese Funktion, um jeden Ihrer Stadt-DataFrames zu reinigen
for city in cleaned_city_list:
    print(city.head())
    print(city.tail())

       Datum   Stadt  Regen  Wind  Sonnenstd  Temp
0 2013-10-03  5100.0    1.1   3.8        6.7  11.5
1 2013-10-04  5100.0   14.0   1.8        1.1  14.0
2 2013-10-05  5100.0    5.9   1.5        0.0  14.2
3 2013-10-06  5100.0    0.0   2.5        1.4  12.4
4 2013-10-07  5100.0    0.0   2.2        1.8  12.1
          Datum   Stadt  Regen  Wind  Sonnenstd  Temp
3648 2023-09-29  5100.0    0.1   3.3      4.233  17.2
3649 2023-09-30  5100.0    0.1   1.2      6.667  13.6
3650 2023-10-01  5100.0    0.0   1.3      8.783  14.6
3651 2023-10-02  5100.0    0.0   1.6      9.367  16.9
3652 2023-10-03  5100.0    5.0   4.4      2.950  15.7
       Datum  Stadt  Regen  Wind  Sonnenstd  Temp
0 2013-10-03   1975    0.0   7.1        9.8   8.2
1 2013-10-04   1975    0.6   6.0        3.6   8.9
2 2013-10-05   1975    0.0   2.2        0.0  12.4
3 2013-10-06   1975    0.0   1.4        0.0  12.8
4 2013-10-07   1975    0.0   1.8        3.7  11.5
          Datum  Stadt  Regen  Wind  Sonnenstd  Temp
3648 2023-09-29  

Nach der erfolgreichen Datenbereinigung werden die bereinigten Daten für jede Stadt in separate CSV-Dateien exportiert. Dies wird mit der Methode to_csv von pandas durchgeführt. Jeder DataFrame aus der Liste cleaned_city_list repräsentiert eine Stadt und wird in einen Dateipfad auf dem Desktop unter dem Verzeichnis Statistical Computing/Fallstudie/ gespeichert. Die Dateinamen entsprechen den Städtenamen.

In [18]:
cleaned_city_list[0].to_csv('./Desktop/Statistical Computing/Fallstudie/trier.csv', index=False)
cleaned_city_list[1].to_csv('./Desktop/Statistical Computing/Fallstudie/hamburg.csv', index=False)
cleaned_city_list[2].to_csv('./Desktop/Statistical Computing/Fallstudie/kiel.csv', index=False)
cleaned_city_list[3].to_csv('./Desktop/Statistical Computing/Fallstudie/dresden.csv', index=False)
cleaned_city_list[4].to_csv('./Desktop/Statistical Computing/Fallstudie/regensburg.csv', index=False)
cleaned_city_list[5].to_csv('./Desktop/Statistical Computing/Fallstudie/rostock.csv', index=False)
cleaned_city_list[6].to_csv('./Desktop/Statistical Computing/Fallstudie/konstanz.csv', index=False)
cleaned_city_list[7].to_csv('./Desktop/Statistical Computing/Fallstudie/bremerhaven.csv', index=False)
cleaned_city_list[8].to_csv('./Desktop/Statistical Computing/Fallstudie/luebeck.csv', index=False)
cleaned_city_list[9].to_csv('./Desktop/Statistical Computing/Fallstudie/freiburg.csv', index=False)
